In [23]:
import collections
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow_text as tf_text
np.set_printoptions(precision=3, suppress=True)


In [ ]:
fileName = '../Data/Reddit_News_DJIA.csv'

# Load CSV data with tf.data

In [ ]:
rawData = tf.data.TextLineDataset(fileName)
rawData.element_spec

# Load CSV data with pandas

In [3]:
redditNewsDf = pd.read_csv(fileName)
redditNewsDf.head(1)
#redditNewsDf.Top1.values[0]

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""


In [4]:
rows, cols = redditNewsDf.shape
cols = cols - 2 # subtract date and label
print('Rows =', rows)
print('Columns =', cols)
N = rows * cols # N = Number of documents
print('N: number of docs = ', N)

Rows = 1989
Columns = 25
N: number of docs =  49725


In [5]:
dfLabels = redditNewsDf.Label.values.reshape(-1,1)
dfLabels[:5]

array([[0],
       [1],
       [0],
       [0],
       [1]], dtype=int64)

In [6]:

#dfFeatures = np.char.encode(redditNewsDf.iloc[:,2:].values, encoding='utf-8')
dfFeatures = redditNewsDf.iloc[:,2:].values
dfFeatures = np.char.strip(np.asarray(dfFeatures,dtype=str),chars='b\'\"')

print(dfFeatures[0])

["Georgia 'downs two Russian warplanes' as countries move to brink of war"
 'BREAKING: Musharraf to be impeached.'
 'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)'
 'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'
 "Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing"
 '150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.'
 "Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side"
 "The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it."
 'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]'
 'Did the U.S. Prep Georgia for War with Russia?'
 'Rice Gives Green 

In [7]:
dfFeatures = dfFeatures.reshape(-1)
dfFeatures.shape

(49725,)

In [8]:
dfLabels = dfLabels * np.ones((rows,cols))
dfLabels = dfLabels.reshape(-1)
dfLabels.shape

(49725,)

# TF

In [57]:
#lines = [dfLabels,dfFeatures]

featuresDS = tf.data.Dataset.from_tensor_slices(list(dfFeatures))
labelDS =  tf.data.Dataset.from_tensor_slices(tf.cast(list(dfLabels), tf.int64))
#linesDS = featuresDS.concatenate(tf.data.Dataset.from_tensor_slices(list(tf.cast(dfLabels,tf.int64))))
#for i,row in featuresDS.enumerate().as_numpy_iterator():
#    featuresDS.map(lambda element: [element, tf.cast(dfLabels[i], tf.int64)] )
labeledDS = tf.data.Dataset.zip((featuresDS, labelDS))
#list(labeledDS.as_numpy_iterator())


[(b"Georgia 'downs two Russian warplanes' as countries move to brink of war",
  0),
 (b'BREAKING: Musharraf to be impeached.', 0),
 (b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)',
  0),
 (b'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire',
  0),
 (b"Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing",
  0),
 (b'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.',
  0),
 (b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side",
  0),
 (b"The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it.",
  0),
 (b'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO

b'BREAKING: Musharraf to be impeached.'

In [14]:
tokenizer = tf_text.UnicodeScriptTokenizer()

In [24]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
VALIDATION_SIZE = 5000
VOCAB_SIZE = 10000

In [17]:
def tokenize(text):
  lower_case = tf_text.case_fold_utf8(text)
  return tokenizer.tokenize(lower_case)

In [18]:
tokenized_ds = featuresDS.map(tokenize)

In [19]:
for text_batch in tokenized_ds.take(5):
  print("Tokens: ", text_batch.numpy())

Tokens:  [b'georgia' b"'" b'downs' b'two' b'russian' b'warplanes' b"'" b'as'
 b'countries' b'move' b'to' b'brink' b'of' b'war']
Tokens:  [b'breaking' b':' b'musharraf' b'to' b'be' b'impeached' b'.']
Tokens:  [b'russia' b'today' b':' b'columns' b'of' b'troops' b'roll' b'into'
 b'south' b'ossetia' b';' b'footage' b'from' b'fighting' b'(' b'youtube'
 b')']
Tokens:  [b'russian' b'tanks' b'are' b'moving' b'towards' b'the' b'capital' b'of'
 b'south' b'ossetia' b',' b'which' b'has' b'reportedly' b'been'
 b'completely' b'destroyed' b'by' b'georgian' b'artillery' b'fire']
Tokens:  [b'afghan' b'children' b'raped' b'with' b"'" b'impunity' b",'" b'u' b'.'
 b'n' b'.' b'official' b'says' b'-' b'this' b'is' b'sick' b',' b'a'
 b'three' b'year' b'old' b'was' b'raped' b'and' b'they' b'do' b'nothing']


In [21]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [25]:
tokenized_ds = configure_dataset(tokenized_ds)

vocab_dict = collections.defaultdict(lambda: 0)
for toks in tokenized_ds.as_numpy_iterator():
  for tok in toks:
    vocab_dict[tok] += 1

vocab = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
vocab = [token for token, count in vocab]
vocab = vocab[:VOCAB_SIZE]
vocab_size = len(vocab)
print("Vocab size: ", vocab_size)
print("First five vocab entries:", vocab[:5])

Vocab size:  10000
First five vocab entries: [b'the', b'to', b'.', b'in', b'of']


# Scikit

In [ ]:
trainPercent = 0.8
dfFeatures_train = dfFeatures[:int(trainPercent*N)]
dfLabels_train = dfLabels[:int(trainPercent*N)]
dfFeatures_test = dfFeatures[int(trainPercent*N):]
dfLabels_test = dfLabels[int(trainPercent*N):]

print('Training features =',dfFeatures_train.shape)
print('Training labels =',dfLabels_train.shape)
print('Testing features =',dfFeatures_test.shape)
print('Testing labels =',dfLabels_test.shape)

In [ ]:
type(np.asarray(dfFeatures,dtype=str))
dfFeatures.shape

In [ ]:
vectorizer = CountVectorizer(max_features=10000)
#vectorizer.fit(np.asarray(dfFeatures_train,dtype=str))
vectorizer.fit(dfFeatures_train)
vocabulary = np.array(list(vectorizer.vocabulary_.items()))
print('First 5 indexed vocab words:\n', vocabulary[:5])
M = vocabulary.shape[0]
print('\nNumber of words =', M)

In [ ]:
docMatrix_train = vectorizer.transform(dfFeatures_train).toarray()
print(dfFeatures_train[0])
print('First 5 rows of Doc Matrix:\n', docMatrix_train[:5])

print('\nDoc Matrix Shape =', docMatrix_train.shape)

In [ ]:
docMatric_test = vectorizer.transform(dfFeatures_test).toarray()

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(tol=0.1, max_iter=500)
classifier.fit(docMatrix_train, dfLabels_train)
score = classifier.score(docMatric_test, dfLabels_test)

print("Accuracy:", score)